In [1]:
# Last run with HEAD at 531f56d40d14deb0336ce7494efcd05c049f71a9

In [2]:
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [3]:
# relative to nplinker source dir
import sys
sys.path.append('..')

In [4]:
import os
import numpy

from nplinker import nplinker
from nplinker import logconfig
logconfig.LogConfig.setLogLevel('DEBUG') # 'INFO' is a lot less spammy
from nplinker.scoring import IOKR


['/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages/cdk_pywrapper']
Server process already running: True


In [5]:
npl = nplinker.NPLinker('carnegie.toml')

10:20:35 [DEBUG] config.py:83, Parsing default config file: /home/grimur/.config/nplinker/nplinker.toml
10:20:35 [DEBUG] config.py:86, Loading user config carnegie.toml
10:20:35 [INFO] config.py:121, Loading from local data in directory /home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/
10:20:35 [DEBUG] loader.py:94, DatasetLoader(/home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/, , False)


In [6]:
npl.load_data()

10:20:36 [DEBUG] nplinker.py:212, load_data (normal case, full load)
10:20:36 [INFO] loader.py:337, Loaded global strain IDs (0 total)
10:20:36 [INFO] loader.py:345, Loaded dataset strain IDs (25 total)
10:20:37 [INFO] metabolomics.py:626, 3107 molecules parsed from MGF file
10:20:37 [DEBUG] metabolomics.py:347, loading edges file: /home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/networkedges_selfloop/455bd15e8bd645789792b347648be818..selfloop [3107 spectra from MGF]
/home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/clusterinfo_summary/6f4bf34641bf4bcf8711042bc8cb086f.tsv False
10:20:37 [INFO] metabolomics.py:646, quantification table exists, new-style GNPS dataset
10:20:37 [INFO] metabolomics.py:592, Merged nodes data (new-style), total lines = 3107
10:20:37 [DEBUG] metabolomics.py:684, make_families: 192 molams + 2030 singletons
10:20:37 [INFO] loader.py:323, Loading provided annotation files (/home/grimur/uist-data/nplinker_datasets/carnegie_nomibi

True

In [7]:
interesting_bgcs = [
"KRD026.Scaffold_10.region001",
"KRD026.Scaffold_13.region001",
"KRD140.Scaffold_2.region002",
"KRD140.Scaffold_3.region001",
"KRD162.Scaffold_13.region003",
"KRD175.Scaffold_6.region001",
"KRD197.Scaffold_2.region002"
]

In [8]:
from nplinker.scoring import misc

In [9]:
links_for_bgcs = {}

for bgc_id in interesting_bgcs:
    link_score_lines = []
    for gcf in npl.gcfs:
        if bgc_id in [x.name for x in gcf.bgcs]:
            for spectrum in npl.spectra:
                score = misc.metcalf_scoring(spectrum, gcf, strains=npl.strains, standardised=True)

                for mf in npl.molfams:
                    if spectrum in mf.spectra:
                        break

                # bgc_id,gcf_id,spectrum_id,#spec,#gcf,#overlap,#strains,score,identical,p_val
                strain_name = bgc_id.split('.')[0]
                overlap = len(set(spectrum.strains.keys()).intersection(gcf.strains))
                info_line = [strain_name, 
                             bgc_id, 
                             gcf.gcf_id, 
                             gcf.product_type, 
                             spectrum.spectrum_id, 
                             len(spectrum.strains), 
                             len(gcf.strains), 
                             overlap, 
                             len(npl.strains), 
                             score]

                link_score_lines.append(info_line)
                
    link_score_lines.sort(key=lambda x: x[9], reverse=True)
    links_for_bgcs[bgc_id] = link_score_lines

In [10]:
links_with_score_count = []

for bgc_id, links in links_for_bgcs.items():
    scores = [x[9] for x in links]
    for i, line in enumerate(links):
        if i == 10:
            break
        current_score = line[9]
        current_score_count = scores.count(current_score)

        links_with_score_count.append(line + [current_score_count])
        print(line)

['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 7701, 1, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 240439, 2, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 240461, 2, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 241050, 4, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 242220, 3, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 242381, 3, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 243020, 4, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 279198, 3, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 298372, 2, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 300125, 2, 1, 1, 17, 4.000000000000001]
['KRD026', 'KRD026.Scaffold_13.region001', 29, 'NRPS

In [11]:
import csv

In [12]:
with open('metcalf_interesting_links_20200212.csv', 'w') as f:
    w = csv.writer(f)
    w.writerow(["# strain_name", 
                 "bgc_id",
                 "gcf_id",
                 "product_type",
                 "spectrum_id",
                 "spectrum_strains",
                 "gcf_strains",
                 "overlap",
                 "total_strains",
                 "score",
                 "score_count"])
    w.writerows(links_with_score_count)

In [23]:
gnps_media_file = "/home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/clusterinfo_summary/6f4bf34641bf4bcf8711042bc8cb086f.tsv"

viable_spectrum_ids = []

with open(gnps_media_file, 'r') as f:
    r = csv.reader(f, delimiter='\t')
    header = next(r)
    # print(header)
    # print(header[21])
    # print(header[24])
    for row in r:
        # print(row)
        medium = row[0]
        spectrum_id = row[21]
        cluster_id = row[24]
        if not "BLANK" in medium:
            viable_spectrum_ids.append(cluster_id)


SpectrumID
cluster index


In [21]:
links_with_score_count_no_blanks = []

for bgc_id, links in links_for_bgcs.items():
    scores = [x[9] for x in links]
    count = 0
    for line in links:
        spectrum_id = line[4]
        if str(spectrum_id) not in viable_spectrum_ids:
            continue
        if count == 10:
            break
        current_score = line[9]
        current_score_count = scores.count(current_score)
        line.append(current_score_count)

        links_with_score_count_no_blanks.append(line + [current_score_count])
        count += 1
        print(line)

['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 240439, 2, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 240461, 2, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 241050, 4, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 242220, 3, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 242381, 3, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 243020, 4, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 279198, 3, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 298372, 2, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 300125, 2, 1, 1, 17, 4.000000000000001, 14, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region

In [22]:
with open('metcalf_interesting_links_no_blanks_20200213.csv', 'w') as f:
    w = csv.writer(f)
    w.writerow(["# strain_name", 
                 "bgc_id",
                 "gcf_id",
                 "product_type",
                 "spectrum_id",
                 "spectrum_strains",
                 "gcf_strains",
                 "overlap",
                 "total_strains",
                 "score",
                 "score_count"])
    w.writerows(links_with_score_count_no_blanks)